<a href="https://colab.research.google.com/github/vipashaaV321/Session-Based-Movie-Recommendation-GRU4RECBE/blob/main/IR_Reranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers

In [3]:
import pandas as pd
import time
from tqdm import tqdm
import seaborn as sns
import numpy as np
from textblob import TextBlob
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')

.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

In [4]:
def fetch_movie_info(dataframe_idx):
    info = df.iloc[dataframe_idx]
    meta_dict = {}
    meta_dict['title'] = info['title']
    meta_dict['plot'] = info['plot'][:500]
    return meta_dict

In [5]:
def search(query, top_k, index, model):
    t=time.time()
    query_vector = model.encode([query])
    top_k = index.search(query_vector, top_k)
    print('>>>> Results in Total Time: {}'.format(time.time()-t))
    top_k_ids = top_k[1].tolist()[0]
    top_k_ids = list(np.unique(top_k_ids))
    results =  [fetch_movie_info(idx) for idx in top_k_ids]
    return results

In [6]:
#ACCESS DATASET FROM GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
datapath= "/content/drive/MyDrive/Data/movie_rating_plot.csv"
data= pd.read_csv(datapath)

In [8]:
data.head()

,Unnamed: 0,userId,movieId,rating,timestamp,title,genres,plot
0,0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,A cowboy doll is profoundly threatened and jea...
1,1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,A cowboy doll is profoundly threatened and jea...
2,2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,A cowboy doll is profoundly threatened and jea...
3,3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,A cowboy doll is profoundly threatened and jea...
4,4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,A cowboy doll is profoundly threatened and jea...


In [9]:
import gc
df = data[['title','plot']]

In [10]:
df.dropna(inplace=True)
df.drop_duplicates(subset=['plot'],inplace=True)

<ipython-input-10-205169fa8e08>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
<ipython-input-10-205169fa8e08>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['plot'],inplace=True)


In [11]:
!pip install faiss

ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss


In [12]:
pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 46.1 MB/s eta 0:00:00


In [14]:
import faiss
encoded_data = model.encode(df['plot'].tolist())
encoded_data = np.asarray(encoded_data.astype('float32'))
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.array(range(0, len(df))))
faiss.write_index(index, 'movie_plot.index')


In [15]:
from pprint import pprint

query="Artificial Intelligence based action movie"
results=search(query, top_k=5, index=index, model=model)

print("\n")
for result in results:
    print('\t',pprint(result))

>>>> Results in Total Time: 0.06105208396911621


{'plot': 'When terrorists seize control of an airliner, an intelligence '
         'analyst accompanies a commando unit for a midair boarding operation.',
 'title': 'Executive Decision (1996)'}
	 None
{'plot': 'IMF agent Ethan Hunt is sent to Sydney to find and destroy a '
         'genetically modified disease called "Chimera".',
 'title': 'Mission: Impossible II (2000)'}
	 None
{'plot': 'An astronaut gets infected with alien DNA during the first mission '
         'on Mars and runs amok on earth. Preston and Laura team up with a '
         'peaceful, genetically re-engineered Sil to track the monster down.',
 'title': 'Species II (1998)'}
	 None
{'plot': "This fourth installment of Chan's Police Story film franchise has "
         'our hero trying to locate a missing nuclear warhead.',
 'title': 'First Strike (Police Story 4: First Strike) (Ging chaat goo si 4: '
          'Ji gaan daan yam mo) (1996)'}
	 None
{'plot': 'Mario Van Peeb

#### Re-Ranker: Cross-Encoder

The retriever has to be efficient for large document collections with millions of entries. However, it might return irrelevant candidates.

A re-ranker based on a Cross-Encoder can substantially improve the final results for the user. The query and a possible document is passed simultaneously to transformer network, which then outputs a single score between 0 and 1 indicating how relevant the document is for the given query.

The advantage of Cross-Encoders is the higher performance, as they perform attention across the query and the document.

Scoring thousands or millions of (query, document)-pairs would be rather slow. Hence, we use the retriever to create a set of e.g. 100 possible candidates which are then re-ranked by the Cross-Encoder.

In [16]:
## Load our cross-encoder. Use fast tokenizer to speed up the tokenization
from sentence_transformers import CrossEncoder
cross_model = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-6', max_length=512)

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [17]:
def cross_score(model_inputs):
    scores = cross_model.predict(model_inputs)
    return scores

model_inputs = [[query,item['plot']] for item in results]
scores = cross_score(model_inputs)
#Sort the scores in decreasing order
ranked_results = [{'Title': inp['title'], 'Score': score} for inp, score in zip(results, scores)]
ranked_results = sorted(ranked_results, key=lambda x: x['Score'], reverse=True)

In [18]:
print("\n")
for result in ranked_results:
    print('\t',pprint(result))



{'Score': 0.00017362922,
 'Title': 'First Strike (Police Story 4: First Strike) (Ging chaat goo si 4: '
          'Ji gaan daan yam mo) (1996)'}
	 None
{'Score': 0.00016650824, 'Title': 'Mission: Impossible II (2000)'}
	 None
{'Score': 0.0001656354, 'Title': 'Executive Decision (1996)'}
	 None
{'Score': 0.00016177178, 'Title': 'Species II (1998)'}
	 None
{'Score': 0.00016062865, 'Title': 'Solo (1996)'}
	 None


In [ ]:
from sklearn.metrics import ndcg_score
import numpy as np

# Assuming you have relevance labels for each result, you can define them as a list.
# For example, if the first result is relevant and the rest are not, you can set labels as [1, 0, 0, ...]
relevance_labels = [1] + [0] * (len(results) - 1)

# Sort the results based on the scores
ranked_results = sorted(ranked_results, key=lambda x: x['Score'], reverse=True)

# Extract the sorted relevance labels
sorted_labels = [relevance_labels[results.index(item)] for item in ranked_results]

# Calculate MRR
def calculate_mrr(sorted_labels):
    for i, label in enumerate(sorted_labels):
        if label == 1:
            return 1 / (i + 1)
    return 0

mrr = calculate_mrr(sorted_labels)

# Calculate NDCG
ndcg = ndcg_score([sorted_labels], [np.ones_like(sorted_labels)])

# Calculate Hit Rate at position 1, 5, and 10
hit_rate_at_1 = sorted_labels[0]
hit_rate_at_5 = any(sorted_labels[:5])
hit_rate_at_10 = any(sorted_labels[:10])

print(f'MRR: {mrr}')
print(f'NDCG: {ndcg}')
print(f'Hit Rate@1: {hit_rate_at_1}')
print(f'Hit Rate@5: {hit_rate_at_5}')
print(f'Hit Rate@10: {hit_rate_at_10}')


In [20]:
!pip install bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.2 MB/s eta 0:00:00


In [21]:
# check your installation
import bert_score
bert_score.__version__

'0.3.12'

In [22]:
from bert_score import score

In [23]:
ref=["Artificial Intelligence based action movie"]

In [25]:
ranked_results_bert = []

for cand in results:
    P, R, F1 = score([cand['plot']], ref, lang='en')
    ranked_results_bert.append({'Title': cand['title'], 'Score': F1.numpy()[0]})

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

In [26]:
#Sort the scores in decreasing order
ranked_results_bert = sorted(ranked_results_bert, key=lambda x: x['Score'], reverse=True)
print("\n")
for result in ranked_results_bert:
    print('\t',pprint(result))



{'Score': 0.8447, 'Title': 'Executive Decision (1996)'}
	 None
{'Score': 0.84184855,
 'Title': 'First Strike (Police Story 4: First Strike) (Ging chaat goo si 4: '
          'Ji gaan daan yam mo) (1996)'}
	 None
{'Score': 0.82105184, 'Title': 'Mission: Impossible II (2000)'}
	 None
{'Score': 0.8170033, 'Title': 'Species II (1998)'}
	 None
{'Score': 0.8047478, 'Title': 'Solo (1996)'}
	 None


In [34]:
[item['title'] for item in results]

['Executive Decision (1996)',
 'Mission: Impossible II (2000)',
 'Species II (1998)',
 'First Strike (Police Story 4: First Strike) (Ging chaat goo si 4: Ji gaan daan yam mo) (1996)',
 'Solo (1996)']

In [37]:
final_results = pd.DataFrame()
final_results['faiss_ranking'] = [item['title'] for item in results]
final_results['cross_encoder'] = [item['Title'] for item in ranked_results]
final_results['bert_score'] = [item['Title'] for item in ranked_results_bert]

In [38]:
final_results

,faiss_ranking,cross_encoder,bert_score
0,Executive Decision (1996),First Strike (Police Story 4: First Strike) (G...,Executive Decision (1996)
1,Mission: Impossible II (2000),Mission: Impossible II (2000),First Strike (Police Story 4: First Strike) (G...
2,Species II (1998),Executive Decision (1996),Mission: Impossible II (2000)
3,First Strike (Police Story 4: First Strike) (G...,Species II (1998),Species II (1998)
4,Solo (1996),Solo (1996),Solo (1996)
